<a href="https://colab.research.google.com/github/NataliaKor/tutorial/blob/main/point_PE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial**: Three ways to do parametere estimation with Machine Learning.  
*Example of Galactic Binary.*

---

## Natalia Korsakova *korsakova@apc.in2p3.fr*

In this tutorial we build a simple point parameter estimation neural network for galactic binaries.


### Steps

1. **Build a set of training waveforms.** We use the FastGB waveforms package.
2. **Build CNN which takes in data and returns values of parameters.** This is usually called point parameter estimation (regression in the field of machine learning).
3. **Train the model from waveform to parameter**,
    - During training, we add noise to waveforms to make simulated data.
5. **Evaluate** on test data.





We will generate waveform with the FastGB GPU accelerate waveform (https://github.com/mikekatz04/GBGPU). Lightweight version of FastGB is available here:


In [ ]:
# Plotting library not included in colab.
!pip install corner

In [ ]:
# Waveform generation libary based on FastGB plus noise curve for LISA.
!git clone https://github.com/NataliaKor/GBGPU.git

In [3]:
from GBGPU.gbgpu.gbgpu import GBGPU
from GBGPU.gbgpu.noisemodel import AnalyticNoise

In [ ]:
# Make the code hardware agnostic.
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
from torch import distributions

import matplotlib.pyplot as plt
import corner

import numpy as np

# Making code agnostic to CPU/GPU
def std_get_wrapper(arg):
    return arg

def cuda_get_wrapper(arg):
    return arg.get()

if torch.cuda.is_available():
   import cupy as cp
   gpu = True
   get_wrapper = cuda_get_wrapper
   dev = "cuda:0"
   dtype = torch.cuda.FloatTensor
else:
   import numpy as cp
   gpu = False
   get_wrapper = std_get_wrapper
   dev = "cpu"
   dtype = torch.FloatTensor

print(gpu)

## Training data

### Signal model

The galactic binary waveforms that we use depend on 8 parameters $\theta$:
* Amplitude `amp`
* Initial frequency `f0` (Hz)
* Initial time derivative of frequency `fdot` (Hz/s)
* Initial phase angle of gravitational wave `phi0` (radians)
* Inclination of the orbit `iota` (radians)
* Ecliptic longitude `lam` (radians)
* Ecliptic lattitude `beta` (radians)
* Polarization angle `psi` (radians)

The FastGB package then gives us waveforms $h(\theta) = (A(\theta), E(\theta))$ where $A$, $E$ are TDI channels.

Initially, we perform inference over `beta` and `lam`, holding the remaining parameters at fixed values. Training data must be drawn from the prior, and we choose uniform `sin(beta)`, `lam` priors over some range.

### Noise

Ultimately we must train on simulated data, which also include noise,
$$
d = h(\theta) + n, \qquad n \sim p_{S_n}(n).
$$
The noise is taken to be stationary Gaussian with some power spectral density $S_n$.

Rather than creating complete simulated data sets in advance of training, **we only prepare the waveforms in advance, and we add noise realizations during training.** The reason for this is that we would like the training dataset to be as large as possible to reduce the risk of overfitting. Noise is fast to sample, so this can be done during training, and doing so effectively makes the training set much larger. Generally waveforms are slower to generate, so general practice is to re-use them in each epoch. However with a very fast waveform model (e.g., using GPU acceleration) training could be further improved by generating all data on the fly.

### Parameter sampling

Initially, we construct a dataset consisting of $10^4$ waveforms, all drawn from the prior.

In [5]:
# Size of the training set.
num_samples = 10000

# We choose a very narrow frequency range.
f0_lower = 0.010062
f0_upper = 0.010084

# Amplitude range.
amp_lower = 1e-23
amp_upper = 1e-21

# Sample f0 and amp from a uniform prior.
f0 = cp.random.uniform(f0_lower, f0_upper, num_samples)
amp = cp.random.uniform(amp_lower, amp_upper, num_samples)


In [6]:
# Fixed parameters
ones = cp.ones(num_samples)
fdot = 1.79e-15 * ones
lam  = 4.36 * ones
beta = 2.18 * ones
iota = 0.67 * ones
phi0 = 5.48 * ones
psi  = 0.43 * ones

In [7]:
# Package parameters into arrays.

sampling_parameters = cp.vstack((f0, amp)).T
all_parameters = cp.vstack((amp, f0, fdot, cp.zeros(num_samples), -phi0, iota, psi, lam, beta)).T

### Waveform generation

In [8]:
# Initialise waveform generator.
gb = GBGPU(use_gpu=gpu)

# Waveform settings
Tobs = 31536000.0  # One-year observation
dt = 15.0  # Sample rate (Nyquist is safely larger than the maximum frequency we will encounter)
df = 1./Tobs
N_points = 128

# Generate the waveforms.
gb.run_wave(*all_parameters.T, N = N_points, dt = dt, T = Tobs, oversample = 1)

The waveforms stored in gb are only defined over very narrow frequency support. They need to be stitched into a wider frequency band, which we define here. We choose a frequency range slightly larger than [f0_lower, f0_upper] such that full waveform can always fit in a band.

In [10]:
f_min = 0.010059
f_max = 0.0100861

# Define the frequency grid.
num_bins = int((f_max - f_min) / df) + 1
sample_frequencies = cp.linspace(f_min, f_max, num=num_bins)

Neural networks perform best when inputs are standardized. For simulated data, we achieve this by whitening the waveforms using the predicted LISA power spectral density.

In [11]:
noise = AnalyticNoise(sample_frequencies)
psd_A, psd_E = noise.psd(option="A"), noise.psd(option="E")

asd_A = cp.sqrt(psd_A)
asd_E = cp.sqrt(psd_E)

In [ ]:
plt.plot(get_wrapper(sample_frequencies), get_wrapper(asd_A))
plt.title('Amplitude spectral density in a band')
plt.xlabel('frequency (Hz)')
plt.ylabel('amplitude spectral density (channel A)')
plt.show()

Finally, we stitch the waveforms into the frequency grid, and at the same time apply the whitening.

In [13]:
k_min = round(f_min/df)
k_max = round(f_max/df)
num = len(sample_frequencies)

# These indices describe how to stitch the waveform into the larger frequency grid.
i_start = (get_wrapper(gb.start_inds) - k_min).astype(cp.int32)
i_end = (get_wrapper(gb.start_inds) - k_min + gb.N).astype(cp.int32)

# PyTorch by default uses float32, and that should be sufficient for our purposes.
# Here we use complex64 since the frequency-domain strain is complex.

A_whitened = cp.empty((num_samples, num), dtype=cp.complex64)
E_whitened = cp.empty((num_samples, num), dtype=cp.complex64)

for i in range(num_samples):
    x = cp.zeros(num, dtype=cp.complex128)
    x[i_start[i]:i_end[i]] = gb.A[i]
    x *= cp.sqrt(4 * df) / asd_A
    A_whitened[i] = x

    x = cp.zeros(num, dtype=cp.complex128)
    x[i_start[i]:i_end[i]] = gb.E[i]
    x *= cp.sqrt(4 * df) / asd_E
    E_whitened[i] = x

In [ ]:
# Plot a sample waveform
plt.plot(get_wrapper(sample_frequencies), get_wrapper(A_whitened[0].real))
plt.xscale('log')
plt.xlabel('$f$')
plt.ylabel('Re $A$')

plt.show()

### Package into a pytorch Dataset

The `Dataset` is a convenient class for storing and accessing pairs of parameters and associated data. It must define the following methods:

* `__len__()`: Return total number of samples in the dataset.
* `__getitem__(idx)`: Retrieve a $(\theta, d)$ pair of parameters and data. We use this method to also add (in real time) a noise realization to each simulated waveform. (Therefore repeated calls will give different noise realizations).


In [15]:
# For best training, parameters should be standardized (zero mean, unit variance across the training set).

parameters_mean = np.mean(sampling_parameters, axis=0)
parameters_std = np.std(sampling_parameters, axis=0)

parameters_standardized = (sampling_parameters - parameters_mean) / parameters_std
parameters_standardized = parameters_standardized.astype(np.float32)

In [ ]:
# Prepare the (complex) frequency-domain data for the (real) neural network. To do so we simply concatenate the real and imaginary parts into an array of doubled length.


waveforms = cp.concatenate((cp.expand_dims(A_whitened, axis=1).real, cp.expand_dims(A_whitened, axis=1).imag,
                            cp.expand_dims(E_whitened, axis=1).real, cp.expand_dims(E_whitened, axis=1).imag),axis=1)

print(waveforms.shape)

In [17]:
class WaveformDataset(Dataset):

    def __init__(self, parameters, waveforms):
        self.parameters = parameters
        self.waveforms = waveforms

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        params = self.parameters[idx]
        signal = self.waveforms[idx]

        # Add unit normal noise to the signal
        noise = cp.random.normal(size = signal.shape).astype(cp.float32)
        data = signal + noise

        return torch.tensor(data).type(dtype), torch.tensor(params).type(dtype)

In [18]:
waveform_dataset = WaveformDataset(parameters_standardized, waveforms)

In [ ]:
# We can sample from the WaveformDataset. This gives us pairs of data and parameters, different noise realizations each time.

x, y = waveform_dataset[0]
print(y)
plt.plot(x[0].cpu())
plt.show()

## Point parameter estimation.

We now try to take the imput data and fit it to the corresponding parameters.

Model construction

We start with the simple CNN model which combines a set of different layers using `torch.nn.Sequential`

In [ ]:
model = torch.nn.Sequential(
          nn.Conv1d(4,16,8,padding="same"), # [input channels, number of filters, filter size]
          nn.BatchNorm1d(16),
          nn.Tanh(),
          nn.MaxPool1d(2),
          nn.Conv1d(16,64,8,padding="same"),
          nn.Tanh(),
          nn.MaxPool1d(2),
          nn.Conv1d(64,8,8,padding="same"),
          nn.Tanh(),
          nn.MaxPool1d(2),
          nn.Flatten(),
          nn.LazyLinear(512), # LazyLinear automatically computes the input size from flattened convolutions
          nn.Sigmoid(),
          nn.LazyLinear(128),
          nn.Sigmoid(),
          nn.LazyLinear(2)
        ).to(dev)


### Training

In [21]:
# Split the dataset into training and test sets. We use the test set to make sure the network properly generalizes to data that it has not seen in training, i.e., it does not overfit.

train_fraction = 0.8
num_train = int(round(train_fraction * num_samples))
num_test = num_samples - num_train
train_dataset, test_dataset = random_split(waveform_dataset, [num_train, num_test])

# The DataLoader is used in training.

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [22]:
# The DataLoaders iterate over samples, returning torch tensors containing a batch of data.

train_features, train_labels = next(iter(train_dataloader))


In [23]:
# We use the Adam optimizer.

optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad==True], lr=1e-4)

Define loss function

In [24]:
# We use Meas Square Error.

loss_func = torch.nn.MSELoss()


In [25]:
# Training and test loops.

def train_loop(dataloader, model, optimizer):

    size = len(dataloader.dataset)
    train_loss = 0

    for batch, (X, y) in enumerate(dataloader):
        outputs = model(X)
        loss = loss_func(outputs, y)

        train_loss += loss.detach().sum()
        loss = loss.mean()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d} samples]")

    average_loss = train_loss.item() / size
    print('Average loss: {:.4f}'.format(average_loss))
    return average_loss


In [26]:
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            outputs = model(X)
            loss = loss_func(outputs, y)
            test_loss += loss.sum()

    test_loss /= size
    print(f"Test loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
epochs = 20
train_history = []
test_history = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss = train_loop(train_dataloader, model, optimizer)
    train_history.append(loss)
    loss = test_loop(test_dataloader, model)
    test_history.append(loss.cpu())
print("Done!")

In [ ]:
epochs = np.arange(1, len(train_history) + 1)
print(type(test_history))
plt.plot(epochs, train_history, label = 'train loss')
plt.plot(epochs, test_history, label = 'test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [29]:
test_features, test_labels = next(iter(test_dataloader))
test_outputs = model(test_features)


In [ ]:
plt.scatter(test_labels[:,0], test_outputs[:,0].detach().numpy(), label = '$f0$')
plt.scatter(test_labels[:,1], test_outputs[:,1].detach().numpy(), label = 'amplitude')
plt.xlabel('True value of parameter (standerdized)')
plt.ylabel('Predicted value of parameter (standerdized)')
plt.legend()
plt.show()

In [ ]:
num_posteriors = 5
num_samples = 10000

for n in range(num_posteriors):
    test_x, test_y = test_dataset[n]

    # Repeat same wf for a number of samples
    wf = torch.tile(test_x, (num_samples, 1))

    # Predict a posterior
    pred_samples = cp.asarray(model(test_features).detach())

    # Undo the standardization
    pred_samples = parameters_std * pred_samples + parameters_mean
    truth = parameters_std * cp.asarray(test_y) + parameters_mean

    # Plot
    corner.corner(get_wrapper(pred_samples), truths=get_wrapper(truth), labels=['$f0$', '$amp$'])
    plt.show()